In [1]:
'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''
from generate_training_validation_data import TextMaskImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

# dimensions of our images.
img_width, img_height = 64, 64
train_data_dir = 'D:/MemeMachine_ProjectData/dataset/training'
validation_data_dir = 'D:/MemeMachine_ProjectData/dataset/validation'
epochs = 50
batch_size = 32

In [8]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=input_shape))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# # model.add(Dropout(0.5)) TODO consider applying dropout

# model.add(Dense(img_width*img_height))
# model.add(Activation('sigmoid'))

model = Sequential()
# model.add(Dense(img_width*img_height))
# model.add(Activation('relu'))
model.add(K.Input(shape=(16,)))
model.add(Dense(img_width*img_height))
model.add(Activation('sigmoid'))

model.compile(loss='MeanSquaredError', # Mean Squared Error should be good for this application because we are really just trying to predict whether or not a given pixel is in the mask
              optimizer='Adam',
              metrics=['accuracy'])

AttributeError: module 'keras.backend' has no attribute 'Input'

In [16]:
import tensorflow as tf

inputs = tf.keras.Input(shape=(img_width, img_height, 3))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss='MeanSquaredError', # Mean Squared Error should be good for this application because we are really just trying to predict whether or not a given pixel is in the mask
              optimizer='Adam',
              metrics=['accuracy'])

In [12]:
partition = dict()
partition['train'] = [] #list of image ids ("i.e., the file name of the image")
partition['validation'] = [] #list of image ids ("i.e., the file name of the image")

import os
#load the data
for file_ in  os.listdir(train_data_dir+"/"):
    partition['train'].append(str(train_data_dir+"/"+file_))
for file_ in  os.listdir(validation_data_dir+"/"):
    partition['validation'].append(str(validation_data_dir+"/"+file_))

nb_train_samples = 1 #len(partition['train'])
nb_validation_samples = 1 #len(partition['validation'])

In [13]:
# Generators
training_generator = TextMaskImageDataGenerator(partition['train'], x_size=img_width, y_size=img_height, batch_size=batch_size)
validation_generator = TextMaskImageDataGenerator(partition['validation'], x_size=img_width, y_size=img_height, batch_size=batch_size)

In [14]:
# X, y = training_generator.__getitem__(0)
# import cv2 as cv
# cv.imshow('X0',X[0])
# cv.imshow('y0',y[0])
# cv.waitKey(0)



__data_gen,  D:/MemeMachine_ProjectData/dataset/training/078387600a377e8c02bc8dcee612a269.jpg
__data_gen,  D:/MemeMachine_ProjectData/dataset/training/733e8ab58461513fb817a3722e83fe2b.jpg
data generated shapes (32, 64, 64, 3) (32, 64, 64, 3)


-1

In [17]:
model.fit(x=training_generator,
    steps_per_epoch=1,
    epochs=1,
    validation_data=validation_generator,
    validation_steps=1,
    use_multiprocessing=True,
    workers=11, verbose=1)

__data_gen,  D:/MemeMachine_ProjectData/dataset/training/078387600a377e8c02bc8dcee612a269.jpg
__data_gen,  D:/MemeMachine_ProjectData/dataset/training/40f06e0eb953b8728910fd8b031b3e10.jpg
data generated shapes (32, 64, 64, 3) (32, 64, 64, 3)
__data_gen,  D:/MemeMachine_ProjectData/dataset/training/3d048b56deb5f26d16fa92e754ea7ce8.jpg
__data_gen,  D:/MemeMachine_ProjectData/dataset/training/0b6e5121263c2b808ae934955b23267f.jpg
data generated shapes (32, 64, 64, 3) (32, 64, 64, 3)
__data_gen,  D:/MemeMachine_ProjectData/dataset/training/00000543_(4).jpg
__data_gen,  D:/MemeMachine_ProjectData/dataset/training/119a6939d9aa34d08d402dbcc666be39.jpg
data generated shapes (32, 64, 64, 3) (32, 64, 64, 3)
__data_gen,  D:/MemeMachine_ProjectData/dataset/training/d3499442d9a6d8679d62506ce7c66b50.jpg
data generated shapes (32, 64, 64, 3) (32, 64, 64, 3)
__data_gen,  D:/MemeMachine_ProjectData/dataset/training/3a7422bf797ff95e8a7d00e8b8c6b247.jpg
data generated shapes (32, 64, 64, 3) (32, 64, 64, 3

ValueError: Could not pack sequence. Structure had 10 elements, but flat_sequence had 4 elements.  Structure: (<generate_training_validation_data.TextMaskImageDataGenerator object at 0x0000025B5CFC5A58>, None, None), flat_sequence: [<tf.Tensor: shape=(32, 64, 64, 3), dtype=uint8, numpy=
array([[[[215, 125,  70],
         [217, 127,  72],
         [216, 129,  73],
         ...,
         [223, 131,  72],
         [222, 130,  71],
         [215, 125,  66]],

        [[216, 126,  71],
         [216, 126,  71],
         [216, 126,  71],
         ...,
         [224, 132,  73],
         [221, 131,  72],
         [219, 129,  70]],

        [[218, 128,  73],
         [215, 125,  70],
         [216, 126,  71],
         ...,
         [222, 132,  75],
         [223, 133,  76],
         [222, 132,  73]],

        ...,

        [[224, 133,  78],
         [223, 132,  77],
         [224, 133,  78],
         ...,
         [223, 135,  81],
         [219, 130,  79],
         [221, 135,  81]],

        [[

In [ ]:
# model.fit(x=training_generator,
#     steps_per_epoch=nb_train_samples // batch_size,
#     epochs=epochs,
#     validation_data=validation_generator,
#     validation_steps=nb_validation_samples // batch_size,
#     use_multiprocessing=True,
#     workers=11, verbose=1)

In [ ]:
model.save_weights('first_try.h5')


In [ ]:
model.build((img_width, img_height, 3))
model.summary()